In [71]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
import pandas as pd
import numpy as np
from sklearn import preprocessing,cross_decomposition,model_selection,metrics
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
%matplotlib inline
import dateutil
from tqdm import tqdm
from sklearn import linear_model
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import tsa
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [73]:
confirmedGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',encoding='utf-8',na_values=None)
confirmedGlobal

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,137853,141489,142414,142414,143183,143439,143439,143439,144285,145008
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,132686,132697,132740,132763,132797,132828,132853,132875,132891,132922
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,153309,154486,155784,157005,158213,159563,160868,162155,163660,165204
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,14273,14359,14379,14379,14464,14498,14498,14498,14577,14586
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,40805,40906,41061,41227,41405,41629,41736,41780,41879,42110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,55845,60180,65607,71144,78269,86957,94913,101173,106347,117121
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,315641,315761,315761,315876,315876,315968,315968,316088,316088,316189
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,6981,6987,6992,6997,7001,7003,7008,7012,7012,7022
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,185649,186279,187602,188573,189731,190615,191527,192071,192316,192956


In [74]:
deathGLobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deathGLobal

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,5983,6213,6266,6266,6325,6357,6357,6357,6477,6515
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,3938,3956,3979,3994,4008,4026,4042,4063,4087,4112
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,957,969,970,973,977,981,982,984,991,994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,254,334,334,370,370,370,370,370,524,524
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,3587,3589,3589,3591,3591,3596,3596,3599,3599,3600
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1368,1370,1371,1371,1372,1372,1373,1373,1373,1374
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,3084,3113,3138,3162,3196,3221,3250,3272,3295,3316


In [75]:
recoverGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recoverGlobal

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,82586
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,130074,130081,130086,130097,130109,130118,130125,130139,130152,130166
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,105604,106337,107041,107776,108537,109349,109951,110577,111322,112050
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,13844,13897,13930,13930,13988,13988,13988,13988,14077,14113
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,34755,34790,34857,34893,35082,35284,35423,35474,35686,35742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,10667,11047,11443,11971,13421,15536,17583,19342,21344,22946
260,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,310525,310601,310601,310961,310961,311380,311380,311526,311526,311560
261,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,4159,4162,4162,4162,4162,4163,4166,4166,4166,4168
262,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,171699,173320,174728,175429,177419,179171,180535,181638,182665,183534


In [76]:
confirmedGlobal['NewContryCode']=confirmedGlobal['Country/Region']+confirmedGlobal['Province/State'].fillna('')
deathGLobal['NewContryCode']=deathGLobal['Country/Region']+deathGLobal['Province/State'].fillna('')
recoverGlobal['NewContryCode']=recoverGlobal['Country/Region']+recoverGlobal['Province/State'].fillna(' ')
recoverGlobal

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,NewContryCode
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,82586,82586,82586,82586,82586,82586,82586,82586,82586,Afghanistan
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,130081,130086,130097,130109,130118,130125,130139,130152,130166,Albania
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,106337,107041,107776,108537,109349,109951,110577,111322,112050,Algeria
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,13897,13930,13930,13988,13988,13988,13988,14077,14113,Andorra
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,34790,34857,34893,35082,35284,35423,35474,35686,35742,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,11047,11443,11971,13421,15536,17583,19342,21344,22946,Vietnam
260,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,310601,310601,310961,310961,311380,311380,311526,311526,311560,West Bank and Gaza
261,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,4162,4162,4162,4162,4163,4166,4166,4166,4168,Yemen
262,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,173320,174728,175429,177419,179171,180535,181638,182665,183534,Zambia


In [77]:
uniqueCOuntryName=pd.unique(confirmedGlobal['Country/Region'])
df2=confirmedGlobal[list(confirmedGlobal.columns[1:2])+list([confirmedGlobal.columns[-2]])]
df2.columns=['Country','values']
dictVal=[]
for i in range(df2.shape[0]):
    dictVal.append(dict(df2.iloc[i]))
df2=df2.sort_values(by='values',ascending=False)
contryNames=list(df2['Country'].values)
(df2.head())

,Country,values
254,US,34603919
147,India,31484605
30,Brazil,19749073
217,Russia,6094379
130,France,5946041


In [78]:
import dateutil
timestamps=[dateutil.parser.parse(i) for i in confirmedGlobal.columns[4:-1]]
datasetsForLine=[]
for i in uniqueCOuntryName:
    temp={}
    temp['label']=i
    temp['data']=confirmedGlobal[confirmedGlobal['Country/Region']==i][confirmedGlobal.columns[4:-1]].sum().values.tolist()
    datasetsForLine.append(temp)

In [79]:
df3=pd.read_json('https://cdn.jsdelivr.net/gh/highcharts/highcharts@v7.0.0/samples/data/world-population-density.json')
df3.head()

,code3,name,value,code
0,ABW,Aruba,582.34,AW
1,AFG,Afghanistan,53.08,AF
2,AGO,Angola,23.11,AO
3,ALB,Albania,104.97,AL
4,AND,Andorra,164.43,AD


In [80]:
dataForMap=[]
for i in uniqueCOuntryName:
    try:
        tempdf=df3[df3['name']==i]
        temp={}
        temp["code3"]=list(tempdf['code3'].values)[0]
        temp["name"]=i
        temp["value"]=df2[df2['Country']==i]['values'].sum()
        temp["code"]=list(tempdf['code'].values)[0]
        dataForMap.append(temp)
    except:
        pass

In [81]:
dataForMap

[{'code3': 'AFG', 'name': 'Afghanistan', 'value': 145008, 'code': 'AF'},
 {'code3': 'ALB', 'name': 'Albania', 'value': 132922, 'code': 'AL'},
 {'code3': 'DZA', 'name': 'Algeria', 'value': 165204, 'code': 'DZ'},
 {'code3': 'AND', 'name': 'Andorra', 'value': 14586, 'code': 'AD'},
 {'code3': 'AGO', 'name': 'Angola', 'value': 42110, 'code': 'AO'},
 {'code3': 'ATG', 'name': 'Antigua and Barbuda', 'value': 1288, 'code': 'AG'},
 {'code3': 'ARG', 'name': 'Argentina', 'value': 4875927, 'code': 'AR'},
 {'code3': 'ARM', 'name': 'Armenia', 'value': 229090, 'code': 'AM'},
 {'code3': 'AUS', 'name': 'Australia', 'value': 33473, 'code': 'AU'},
 {'code3': 'AUT', 'name': 'Austria', 'value': 656949, 'code': 'AT'},
 {'code3': 'AZE', 'name': 'Azerbaijan', 'value': 341183, 'code': 'AZ'},
 {'code3': 'BHR', 'name': 'Bahrain', 'value': 268731, 'code': 'BH'},
 {'code3': 'BGD', 'name': 'Bangladesh', 'value': 1194752, 'code': 'BD'},
 {'code3': 'BRB', 'name': 'Barbados', 'value': 4359, 'code': 'BB'},
 {'code3': 'B

In [82]:
df3=confirmedGlobal[list(confirmedGlobal.columns[1:2])+list(list(confirmedGlobal.columns.values)[-6:-1])]
df3.head()

,Country/Region,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21
0,Afghanistan,143439,143439,143439,144285,145008
1,Albania,132828,132853,132875,132891,132922
2,Algeria,159563,160868,162155,163660,165204
3,Andorra,14498,14498,14498,14577,14586
4,Angola,41629,41736,41780,41879,42110


In [83]:
dataForheatMap=[]
for i in contryNames:
    try:
        tempdf=df3[df3['Country/Region']==i]
        temp={}
        temp["name"]=i
        temp["data"]=[{'x':j,'y':k} for j,k in zip(tempdf[tempdf.columns[1:]].sum().index,tempdf[tempdf.columns[1:]].sum().values)]
        dataForheatMap.append(temp)
    except:
        pass
dataForheatMap

[{'name': 'US',
  'data': [{'x': '7/23/21', 'y': 34400680},
   {'x': '7/24/21', 'y': 34428075},
   {'x': '7/25/21', 'y': 34444083},
   {'x': '7/26/21', 'y': 34533179},
   {'x': '7/27/21', 'y': 34603919}]},
 {'name': 'India',
  'data': [{'x': '7/23/21', 'y': 31293062},
   {'x': '7/24/21', 'y': 31371901},
   {'x': '7/25/21', 'y': 31411262},
   {'x': '7/26/21', 'y': 31440951},
   {'x': '7/27/21', 'y': 31484605}]},
 {'name': 'Brazil',
  'data': [{'x': '7/23/21', 'y': 19632443},
   {'x': '7/24/21', 'y': 19670534},
   {'x': '7/25/21', 'y': 19688663},
   {'x': '7/26/21', 'y': 19707662},
   {'x': '7/27/21', 'y': 19749073}]},
 {'name': 'Russia',
  'data': [{'x': '7/23/21', 'y': 6002303},
   {'x': '7/24/21', 'y': 6025698},
   {'x': '7/25/21', 'y': 6049215},
   {'x': '7/26/21', 'y': 6071893},
   {'x': '7/27/21', 'y': 6094379}]},
 {'name': 'France',
  'data': [{'x': '7/23/21', 'y': 6015469},
   {'x': '7/24/21', 'y': 6041146},
   {'x': '7/25/21', 'y': 6056388},
   {'x': '7/26/21', 'y': 6061802},
  

In [84]:
countryDataSpe=pd.DataFrame(confirmedGlobal[confirmedGlobal['Country/Region']=='China'][confirmedGlobal.columns[4:-1]].sum()).reset_index()
countryDataSpe.columns=['country','values']
countryDataSpe['lagVal']=countryDataSpe['values'].shift(1).fillna(0)
countryDataSpe['incrementVal']=countryDataSpe['values']-countryDataSpe['lagVal']
countryDataSpe['rollingMean']=countryDataSpe['incrementVal'].rolling(window=4).mean()
countryDataSpe=countryDataSpe.fillna(0)
datasetsForLine=[{'label':'Daily Cumulated Data','data':countryDataSpe['values'].values.tolist()},
                {'label':'Rolling Mean 4 days','data':countryDataSpe['rollingMean'].values.tolist()}]

In [85]:
countryDataSpe=pd.DataFrame(confirmedGlobal[confirmedGlobal['Country/Region']=='China'][confirmedGlobal.columns[4:-1]].sum()).reset_index()
countryDataSpe.columns=['country','values']

In [86]:
confirmedGlobal

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,NewContryCode
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,141489,142414,142414,143183,143439,143439,143439,144285,145008,Afghanistan
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,132697,132740,132763,132797,132828,132853,132875,132891,132922,Albania
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,154486,155784,157005,158213,159563,160868,162155,163660,165204,Algeria
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,14359,14379,14379,14464,14498,14498,14498,14577,14586,Andorra
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,40906,41061,41227,41405,41629,41736,41780,41879,42110,Angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,60180,65607,71144,78269,86957,94913,101173,106347,117121,Vietnam
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,315761,315761,315876,315876,315968,315968,316088,316088,316189,West Bank and Gaza
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,6987,6992,6997,7001,7003,7008,7012,7012,7022,Yemen
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,186279,187602,188573,189731,190615,191527,192071,192316,192956,Zambia


In [87]:
recoverGlobal1=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recoverGlobal1['NewContryCode']=recoverGlobal1['Country/Region']+recoverGlobal1['Province/State'].fillna(' ')
recoverGlobal1.dropna( how='any', inplace=False)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,NewContryCode
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,121,121,121,121,121,121,121,121,121,AustraliaAustralian Capital Territory
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AustraliaNew South Wales
10,Northern Territory,Australia,-12.463400,130.845600,0,0,0,0,0,0,...,185,185,186,188,188,188,188,189,190,AustraliaNorthern Territory
11,Queensland,Australia,-27.469800,153.025100,0,0,0,0,0,0,...,1698,1701,1701,1707,1707,1715,1715,1715,1717,AustraliaQueensland
12,South Australia,Australia,-34.928500,138.600700,0,0,0,0,0,0,...,814,816,822,822,822,822,822,826,827,AustraliaSouth Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Gibraltar,United Kingdom,36.140800,-5.353600,0,0,0,0,0,0,...,4299,4302,4319,4336,4336,4371,4374,4393,4403,United KingdomGibraltar
250,Isle of Man,United Kingdom,54.236100,-4.548100,0,0,0,0,0,0,...,1623,1623,1623,1638,1638,1686,1686,1686,1686,United KingdomIsle of Man
251,Montserrat,United Kingdom,16.742498,-62.187366,0,0,0,0,0,0,...,19,19,19,19,19,19,19,19,19,United KingdomMontserrat
252,"Saint Helena, Ascension and Tristan da Cunha",United Kingdom,-7.946700,-14.355900,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,"United KingdomSaint Helena, Ascension and Tris..."


In [88]:
x=input("Enter a country: ")

In [89]:
x

''

In [90]:
final=recoverGlobal1[recoverGlobal1['Province/State']==x]
final

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,NewContryCode


In [92]:
data=[['1', final.iloc[:,-11].astype(int)], ['2', final.iloc[:,-10].astype(int)], ['3', final.iloc[:,-9].astype(int)], ['4', final.iloc[:,-8].astype(int)], ['5', final.iloc[:,-7].astype(int)], ['6', final.iloc[:,-6].astype(int)], ['7', final.iloc[:,-5].astype(int)], ['8', final.iloc[:,-4].astype(int)], ['9', final.iloc[:,-3].astype(int)], ['10', final.iloc[:,-2].astype(int)] ]

TypeError: cannot convert the series to <class 'int'>

In [ ]:
df = pd.DataFrame(data, columns = ['Day', 'Cases']) 
df

,Day,Cases
0,1,107
1,2,107
2,3,133
3,4,146
4,5,146
5,6,147
6,7,147
7,8,147
8,9,147
9,10,179


In [70]:
x =np.asarray(df['Day']).reshape(-1,1)
y =np.asarray(df['Cases']).reshape(-1,1)

NameError: name 'df' is not defined

In [121]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [122]:
model = LinearRegression()
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [123]:
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [124]:
predict_test = model.predict(x_test)

In [125]:
#(predict_test)

In [126]:
model.predict([[11]])

array([[167.98648649]])